In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('jobs.csv')

In [3]:
df.columns

Index(['url', 'desc'], dtype='object')

In [4]:
df.head()

,url,desc
0,https://www.naukri.com/job-listings-software-e...,{'General Description': 'As a Software Enginee...
1,https://www.naukri.com/job-listings-senior-ful...,"{'General Description': ""Position Overview\nTh..."
2,https://www.naukri.com/job-listings-perl-devel...,{'General Description': 'Greetings from Citius...
3,https://www.naukri.com/job-listings-python-aut...,{'General Description': 'Job Title:\nPython Au...
4,https://www.naukri.com/job-listings-data-engin...,"{'Join us as a Data Engineer, PySpark': 'This ..."


In [5]:
def build_prompt(job_text):
  resume_text = """
SUMMARY
Proficient software engineer with 3 years of experience in developing Python Systems or Services. Experienced in enhancing applications and managing full-cycle deployments using Azure DevOps. Currently exploring AI-based solutions and automation tools like n8n, OpenAI to drive continuous improvement and efficiency.

PROFESSIONAL EXPERIENCE
ARM Embedded Technologies | Sept 2024 - Present  | Support Specialist: Digital Platform Full Stack
- Developed a service in Python to update and validate user details across multiple systems.
- Built Python scripts for bulk user detail updates and data integrity checks.
- Created many alerts in Azure using KQL.
- Automated monitoring reports using Azure Automation with Python.
- Deployed application code to production and pre-production environments using Azure DevOps.
- Implemented Power BI dashboard to get an overview of application performance.
- Worked on UI enhancements and feature development using React JS.

Tata Consultancy Services | Systems Engineer | Oct 2021 - Sept 2024
- Created numerous file-processing and database services using Python based on business needs.
- Scheduled cron jobs in Azure DevOps using Python.
- Implemented new features and functionalities in React JS.
- Built dashboards for application performance monitoring using React JS.
- Managed deployment of code to production environments using Azure DevOps.
- Analyzed reports, identified root causes, monitored application stability, and addressed bugs.
- Used KQL extensively to analyze logs and diagnose system issues.
- Awarded Star of the Quarter (Oct 2022). Top 10 in Tata Neu app ideathon.

TECHNICAL SKILLS
- Programming: Python, JavaScript
- Web: HTML5, CSS, React JS, Flask
- Database: MySQL, PL/SQL (basic)
- Competencies: Problem-Solving, Data Structures & Algorithms, Troubleshooting, System Monitoring, Unit Testing
- Methodologies: Agile, DevOps, CI/CD

EDUCATION
SJB Institute of Technology | B.E. in Computer Science | 8.38 CGPA | Aug 2017 - Aug 2021
"""
  return f"""
You are an expert career assistant.

Given:
1. A candidate's resume
2. A job description in json format

Assess how well the resume matches the job description and return a [JSON object ONLY in this format]:
{{
  "score": float (between 0 and 1),
  "fit": "Good" or "Bad",
  "reason": "Brief reason for the match"
}}

Resume:
{resume_text}

Job Description:
{job_text}
"""

In [14]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai import types # Keep this for GenerateContentConfig
from google.generativeai.types import GenerationConfig

load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
def get_match(job_desc: str):
    prompt = build_prompt(job_desc)
    
    # Directly get the model instance
    model = genai.GenerativeModel("gemini-2.0-flash-lite")

    response = model.generate_content(
        contents=prompt
        )
    
    return response.text

In [ ]:

def get_job_match_score():
    import requests
    import json
    df['score'] = ''
    df['fit'] = 'no idea'
    df['reason'] =''
    for idx, row in df[:1].iterrows():
        res = get_match(row['desc'])
        # print(res)
        try:
        # Safely extract JSON substring
            json_part = res[res.find('{'):res.rfind('}')+1]
            data = json.loads(json_part)
            df.at[idx, 'score'] = data.get('score', -1)
            df.at[idx, 'match'] = data.get('fit', '')
            df.at[idx, 'reason'] = data.get('reason', '')
        except Exception as e:
            print(f"Error at row {idx}: {e}")     



```json
{
  "score": 0.75,
  "fit": "Good",
  "reason": "The candidate has 3 years of Python experience, React experience, and Azure DevOps experience which directly matches the job description. The candidate also demonstrates experience with CI/CD and Agile methodologies, which are also key requirements. The candidate's skills also align with many of the preferred qualifications."
}
```


In [16]:
df[:11]

,url,desc,score,fit,reason,match
0,https://www.naukri.com/job-listings-software-e...,{'General Description': 'As a Software Enginee...,0.75,no idea,The candidate has 3 years of Python experience...,Good
1,https://www.naukri.com/job-listings-senior-ful...,"{'General Description': ""Position Overview\nTh...",,no idea,,NaN
2,https://www.naukri.com/job-listings-perl-devel...,{'General Description': 'Greetings from Citius...,,no idea,,NaN
3,https://www.naukri.com/job-listings-python-aut...,{'General Description': 'Job Title:\nPython Au...,,no idea,,NaN
4,https://www.naukri.com/job-listings-data-engin...,"{'Join us as a Data Engineer, PySpark': 'This ...",,no idea,,NaN
5,https://www.naukri.com/job-listings-software-e...,{'General Description': 'As a Software Enginee...,,no idea,,NaN
6,https://www.naukri.com/job-listings-applied-ai...,{'General Description': 'Be part of a dynamic ...,,no idea,,NaN
7,https://www.naukri.com/job-listings-software-e...,{'General Description': 'As a Software Enginee...,,no idea,,NaN
8,https://www.naukri.com/job-listings-software-d...,{'Position Overview:': 'As an SDE II at Swiggy...,,no idea,,NaN
9,https://www.naukri.com/job-listings-support-en...,{},,no idea,,NaN


In [ ]:
from google import genai

def get_match(resume: str, job_desc: str):
    prompt = build_prompt(resume, job_desc)
    response = genai.Client().models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=[{"text": prompt}],
        # Optional: disable deep thinking to save tokens
        # config=genai.types.GenerateContentConfig(thinking_config=genai.types.ThinkingConfig(thinking_budget=0)),
    )
    return response.text


In [ ]:
import json
import pandas as pd

# Example resume text
resume_text = """..."""  # Replace with your rolled-up resume

df['score'] = ''
df['match'] = ''
df['reason'] = ''

for idx, row in df.iterrows():
    res = get_match(resume_text, row['desc'])
    try:
        # Safely extract JSON substring
        json_part = res[res.find('{'):res.rfind('}')+1]
        data = json.loads(json_part)
        df.at[idx, 'score'] = data.get('score', -1)
        df.at[idx, 'match'] = data.get('fit', '')
        df.at[idx, 'reason'] = data.get('reason', '')
    except Exception as e:
        print(f"Error at row {idx}: {e}")
